In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

ServiceKey ='bNtzO%2Fgs15iHRgzi0jdPCc8ezvYSTV%2FUhcmh9OQOpe4mawtK6mpPj9LiyyVxID%2B20LmTWmLnUP0SAsC3FY3kfg%3D%3D'

#step1---
def main():
  result = []  #국가코드 입력변수
  natName=''
  print("국내 입국한 외국인의 통계 데이터를 수집합니다!")
  nat_cd = input("국가코드를 입력하시오. [중국:112, 일본:130, 미국 275] : ") #국가코드 입력받기_national code
  nStartYear = int(input("데이터를 몇년도 부터 수집할까요? "))
  nEndYear = int(input("데이터를 몇년도 까지 수집할까요?"))
  ed_cd = "E"  #E: 방한외래관광객, D: 해외 출국자 관련 변수

  result, natName, ed, dataEND = getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear) #2번째 함수 호출_결과 얻기, 조회 국가의 데이터
#setp1 finsihed

#step7
  if(natName=='') : #url요청은 성공했지만, 데이터 제공이 안 된 경우
      pirnt("데이터가 전달되지 않았으니, 확인바랍니다.")
  else: 
    #CSV 형태의 파일로 전달
    columns = ["입국자 국가", "국가코드", "입국연월", "입국자 수"]
    result_df = pd.DataFrame(result, columns = columns ) #pandas 라이브러리의 데이타프레임기능활용.
    result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear, dataEND),index=False, encoding ='cp949') #csv 파일의 이름 지정 관련 코드

#setp2 ---
def getTourismStatsService(nat_cd, ed_cd,nStartYear, nEndYear):
    result =[]
    natName=''
    dataEND="{0}{1:0>2}".format(str(nEndYear), str(12)) # 데이터 끝 초기화!_{ } => 인덱스 번호 지정
    isDataEnd = 0 #데이터 끝을 확인하기 위한 flag 초기화! _ 값이끝날 경우, 1로 전환됨.

    for year in range(nStartYear, nEndYear+1): #시작 및 종료 시점 정리 
        for month in range(1,13):
          if(isDataEnd == 1): break #데이터의 끝//flag 설정되어있으면 작업이 중지되어야함.
          yyyymm = "{0}{1:0>2}" .format(str(year),str(month))
          jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd) #3번째 함수 호출 _연도,국가코드, 외국인 구분
#step6() start 
          if (jsonData['response']['header']['resultMsg'] == 'OK'): 
            #입력된 범위까지 수집하지 않았지만, 더 이상 제공되는 데이터가 없는 마지막 항목의 경우
             if jsonData['response']['body']['items'] == '':
                 isDataEnd = 1 #데이터 끝의 flag 설정 
                 dataEND ="{0}{1:0>2}".format(str(year),str(month-1))
                 print("데이터 없음 \n 제공되는 통계데이터는 %s년 %s월 까지 입니다. "%(str(year),str(month -1)))
                 break 
            #jsonData를 출력하여 확인 -------------
             print(json.dumps(jsonData, sort_keys=True,ensure_ascii=False)) #indent = 하위레벨 출력할때 들여쓰기 정도 ,sort_keys =ture 이면 정렬
             # // 아스키코드가 false인 이유 utf-8활용을 위함.
             natName = jsonData['response']['body']['items']['item']['natKorNm'] 
             natName = natName.replace(' ',' ') #국가이름
             num = jsonData['response']['body']['items']['item']['num']
             ed = jsonData['response']['body']['items']['item']['ed']
             print('[ %s_%s : %s]' % (natName,yyyymm, num))
             print('----------------------------------------')
             result.append([natName, nat_cd, yyyymm, num])
        return (result, natName,ed, dataEND) #return 값을 가지고 setp1으로 복귀

#setp3() --
def getTourismStatsItem(yyyymm, national_code, ed_cd) : #요청변수부분
    service_url="http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json&serviceKey=" + ServiceKey #인증키 관련 부분
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + national_code
    parameters += "&ED_CD=" + ed_cd
    url = service_url + parameters

    retData = getRequestUrl(url) # 4번째 함수 호출
    #총 url = http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList+SK+검색조건

    if (retData == None):
        return None
    else:
        return json.loads(retData) #return 값을 가지고, setp2()로 복귀! json데이터=> 파이썬 형태로 변환

#step4() ---
def getRequestUrl(url):
    req = urllib.request.Request(url) # 우리가 생성한 주소로 요청매세지를 생성하여, 전달한다.
    try:
        response = urllib.request.urlopen(req) #서버에 요청 매세지를 보내고, 이에 대한 응답 반환, 200 , 결과값
        if response.getcode() == 200:
            print("[%s] Url Request Sucess" % datetime.datetime.now())
            return response.read().decode('utf-8') #return 값을 가지고, step 3로 복귀!
    except Exception as e:
        print(e)
        print("[%s] Error for URl : %s" % (datetime.datetime.now().url))
        return None

#setp3 finished
#step5() --- 


#step 0 ---
#파이썬에는 main()함수로부터 시작하려는 규칙이 없음
#0level 함수[들여쓰기 안 한 함수]가 먼저시작됨으로, 아래 문장을 작성해주어야 main부터 시작할 수 있다.

if __name__=='__main__':
    main()




국내 입국한 외국인의 통계 데이터를 수집합니다!
국가코드를 입력하시오. [중국:112, 일본:130, 미국 275] : 275
데이터를 몇년도 부터 수집할까요? 2016
데이터를 몇년도 까지 수집할까요?2020
[2022-04-03 21:32:47.219312] Url Request Sucess
{"response": {"body": {"items": {"item": {"ed": "방한외래관광객", "edCd": "E", "natCd": 275, "natKorNm": "미  국", "num": 55739, "rnum": 1, "ym": 201601}}, "numOfRows": 10, "pageNo": 1, "totalCount": 1}, "header": {"resultCode": "0000", "resultMsg": "OK"}}}
[ 미  국_201601 : 55739]
----------------------------------------
[2022-04-03 21:32:47.575017] Url Request Sucess
{"response": {"body": {"items": {"item": {"ed": "방한외래관광객", "edCd": "E", "natCd": 275, "natKorNm": "미  국", "num": 50705, "rnum": 1, "ym": 201602}}, "numOfRows": 10, "pageNo": 1, "totalCount": 1}, "header": {"resultCode": "0000", "resultMsg": "OK"}}}
[ 미  국_201602 : 50705]
----------------------------------------
[2022-04-03 21:32:47.949479] Url Request Sucess
{"response": {"body": {"items": {"item": {"ed": "방한외래관광객", "edCd": "E", "natCd": 275, "natKorNm": "미  국", "num":

In [ ]:
끄